# python 操作 postgresql

## 数据库连接

### 2.1 安装 psycopg2

```bash
lsof -i :8032
kill -9 12345

kubectl get namespaces
kubectl get pods
kubectl --namespace default port-forward pods/postgres-59f958ccd8-j6mbq 8032:5432&

pip3 install psycopg2 -i https://pypi.tuna.tsinghua.edu.cn/simple

# 如果 报错 pg_config executable not found
# sudo apt-get install libpq-dev python-dev
brew install postgresql
# pip3 install psycopg2-binary

psql -h localhost -U postgres --password -p 8032 postgresdb

CREATE DATABASE python;

# 连接成功即可执行查询
# SELECT * FROM pg_database;
```

### 2.2 连接数据库

每条完整的 sql 执行步骤如下，读者应谨记；

1. 建立连接获得 connect 对象
2. 获得游标对象，一个游标对象可以对数据库进行执行操作，非线程安全，多个应用会在同一个连接种创建多个光标；
3. 书写sql语句
4. 调用execute()方法执行sql
5. 抓取数据（可选操作）
6. 提交事物
7. 关闭连接

In [2]:
import psycopg2
# 获得连接
conn = psycopg2.connect(database="python", user="postgres", password="password", host="127.0.0.1", port="8032")
# 获得游标对象
cursor = conn.cursor()
# sql语句
sql = "SELECT VERSION()"
# 执行语句
cursor.execute(sql)
# 获取单条数据.
data = cursor.fetchone()
# 打印
print("database version : %s " % data)
# 事物提交
conn.commit()
# 关闭数据库连接
conn.close()

database version : PostgreSQL 11.15 (Debian 11.15-1.pgdg90+1) on x86_64-pc-linux-gnu, compiled by gcc (Debian 6.3.0-18+deb9u1) 6.3.0 20170516, 64-bit 


## 创建表

### 插入数据姿势一

创建学生表主要有字段id 唯一标识，字段 num 代表学号，字段 name 代表学生姓名；详细的建表默认规则转换见附录

In [3]:
import psycopg2
# 获得连接
conn = psycopg2.connect(database="python", user="postgres", password="password", host="127.0.0.1", port="8032")
# 获得游标对象，一个游标对象可以对数据库进行执行操作
cursor = conn.cursor()
# sql语句 建表
sql = """CREATE TABLE student (
id serial4 PRIMARY KEY, 
num int4,
name varchar(25));"""
# 执行语句
cursor.execute(sql)
print("student table created successfully")
# 事物提交
conn.commit()
# 关闭数据库连接
conn.close()

student table created successfully


## 插入操作

提供的第一种防止sql注入的插入数据方式（具有占位符的预编译sql），重要程度不言而喻；美中不足是字符串类型必须带上单引号；

In [4]:
# -*- coding: utf-8 -*-
import psycopg2
# 获得连接
conn = psycopg2.connect(database="python", user="postgres", password="password", host="127.0.0.1", port="8032")
# 获得游标对象，一个游标对象可以对数据库进行执行操作
cursor = conn.cursor()
# sql语句 建表
sql ="INSERT INTO student (num, name) \
                    VALUES (%s, '%s')" % \
                    (100, 'zszxz')
# 执行语句
cursor.execute(sql)
print("successfully")
# 事物提交
conn.commit()
# 关闭数据库连接
conn.close()

successfully


### 插入数据姿势二（参数分离）

下面参数与 sql 语句分离插入的姿势更简便帅气，也是防止sql注入问题；强烈推荐；

In [5]:
# -*- coding: utf-8 -*-
import psycopg2
# 获得连接
conn = psycopg2.connect(database="python", user="postgres", password="password", host="127.0.0.1", port="8032")
# 获得游标对象，一个游标对象可以对数据库进行执行操作
cursor = conn.cursor()
# sql语句 建表
sql ="""INSERT INTO student (num, name) VALUES (%s, %s)"""
params = (101, 'zszxz')
# 执行语句
cursor.execute(sql,params)
print("successfully")
# 事物提交
conn.commit()
# 关闭数据库连接
conn.close()

successfully


### 插入数据姿势三（字典）

第三种姿势也就是是支持字典映射关系插入，使用字典方式的插入数据是根据字典的key进行匹配占位符，强烈推荐；

In [ ]:
# -*- coding: utf-8 -*-
import psycopg2
# 获得连接
conn = psycopg2.connect(database="python", user="postgres", password="password", host="127.0.0.1", port="8032")
# 获得游标对象，一个游标对象可以对数据库进行执行操作
cursor = conn.cursor()
# sql语句 建表
sql ="""INSERT INTO student (num, name) VALUES (%(num)s, %(name)s)"""
params = {'num':102, 'name':'zszxz'}
# 执行语句
cursor.execute(sql,params)
print("successfully")
# 事物提交
conn.commit()
# 关闭数据库连接
conn.close()

## 查询操作

### 5.1 查询一条数据

使用 fetchone() 方法可以抓取一条数据, 返回的是元组；

In [6]:
# -*- coding: utf-8 -*-
import psycopg2
# 获得连接
conn = psycopg2.connect(database="python", user="postgres", password="password", host="127.0.0.1", port="8032")
# 获得游标对象，一个游标对象可以对数据库进行执行操作
cursor = conn.cursor()
# sql语句 建表
sql ="""SELECT * FROM student;"""
# 执行语句
cursor.execute(sql)
# 抓取
row = cursor.fetchone()
print(row)
# 事物提交
conn.commit()
# 关闭数据库连接
cursor.close()
conn.close()

(1, 100, 'zszxz')


### 查询多条数据

1. 使用 fetchmany([size=cursor.arraysize]) 方法可以抓取多条数据；
2. 此方法可以多次使用，直到数据库中没有数据，此时会返回空列表；
3. 如果不传参数，会限制查询条数，一般就是返回第一条；

In [7]:
# -*- coding: utf-8 -*-
import psycopg2
# 获得连接
conn = psycopg2.connect(database="python", user="postgres", password="password", host="127.0.0.1", port="8032")
# 获得游标对象，一个游标对象可以对数据库进行执行操作
cursor = conn.cursor()
# sql语句 建表
sql ="""SELECT * FROM student;"""
# 执行语句
cursor.execute(sql)
# 抓取
#row = cursor.fetchone()
rows = cursor.fetchmany(2)
print(rows)
# 事物提交
conn.commit()
# 关闭数据库连接
cursor.close()
conn.close()

[(1, 100, 'zszxz'), (2, 101, 'zszxz')]


### 查询全部数据

使用 fetchall() 方法会抓取所有数据；

In [ ]:
# -*- coding: utf-8 -*-
import psycopg2
# 获得连接
conn = psycopg2.connect(database="python", user="postgres", password="123456", host="127.0.0.1", port="5432")
# 获得游标对象，一个游标对象可以对数据库进行执行操作
cursor = conn.cursor()
# sql语句 建表
sql ="""SELECT * FROM student;"""
# 执行语句
cursor.execute(sql)
# 抓取
rows = cursor.fetchall()
print(rows)
# 事物提交
conn.commit()
# 关闭数据库连接
cursor.close()
conn.close()

### 按条件查询

1. 带参查询读者应该谨记sql 与 参数 分离
2. 参数的末尾必须加上逗号
3. 如果知道返回的数据就一条使用fetchone()方法，如果无特殊要求，否则建议使用 fetchall()方法

In [8]:
# -*- coding: utf-8 -*-
import psycopg2
# 获得连接
conn = psycopg2.connect(database="python", user="postgres", password="password", host="127.0.0.1", port="8032")
# 获得游标对象，一个游标对象可以对数据库进行执行操作
cursor = conn.cursor()
# sql语句 建表
sql ="""SELECT * FROM student where id = %s;"""
params = (1,)
# 执行语句
cursor.execute(sql,params)
# 抓取
rows = cursor.fetchall()
print(rows)
# 事物提交
conn.commit()
# 关闭数据库连接
cursor.close()
conn.close()

[(1, 100, 'zszxz')]


## 更新操作

更新操作跟之前的查询，插入类似，参数对应的文章分清楚即可。

In [9]:
# -*- coding: utf-8 -*-
import psycopg2
# 获得连接
conn = psycopg2.connect(database="python", user="postgres", password="password", host="127.0.0.1", port="8032")
# 获得游标对象，一个游标对象可以对数据库进行执行操作
cursor = conn.cursor()
# sql语句 建表
sql ="""update student set name = %s where id = %s  """
params = ('alphazz',3,)
# 执行语句
cursor.execute(sql,params)
# 事物提交
conn.commit()
# 关闭数据库连接
cursor.close()
conn.close()

## 删除操作

删除操作很简单，看如下代码，与之前的代码流程没什么区别；

In [10]:
# -*- coding: utf-8 -*-
import psycopg2
# 获得连接
conn = psycopg2.connect(database="python", user="postgres", password="password", host="127.0.0.1", port="8032")
# 获得游标对象，一个游标对象可以对数据库进行执行操作
cursor = conn.cursor()
# sql语句 建表
sql ="""delete from  student where id = %s  """
params = (3,)
# 执行语句
cursor.execute(sql,params)
# 事物提交
conn.commit()
# 关闭数据库连接
cursor.close()
conn.close()

## 异常处理

处理 sql 的异常非常重要，知识追寻者这边使用 psycopg2 的 Error 进行异常捕获，能捕获到sql 执行时期的所有异常；下面代码中表 test 是库中不存的表，执行sql后会报异常，经过异常捕获后非常美观，不影响程序运行；

In [11]:
# -*- coding: utf-8 -*-
import psycopg2
# 获得连接
conn = psycopg2.connect(database="python", user="postgres", password="password", host="127.0.0.1", port="8032")
# 获得游标对象，一个游标对象可以对数据库进行执行操作
cursor = conn.cursor()
# sql语句 建表
sql ="""select * from test"""
params = (3,)
try:
    # 执行语句
    cursor.execute(sql,params)
except psycopg2.Error as e:
    print(e)
# 事物提交
conn.commit()
# 关闭数据库连接
cursor.close()
conn.close()

relation "test" does not exist
LINE 1: select * from test
                      ^



# 打印sql

使用cursor.query 可以查看执行的sql语句，方便排查；

In [12]:
# -*- coding: utf-8 -*-
import psycopg2
# 获得连接
conn = psycopg2.connect(database="python", user="postgres", password="password", host="127.0.0.1", port="8032")
# 获得游标对象，一个游标对象可以对数据库进行执行操作
cursor = conn.cursor()
# sql语句 建表
sql ="""select * from student"""
try:
    # 执行语句
    cursor.execute(sql,)
    que = cursor.query
    print(que)
except psycopg2.Error as e:
    print(e)
# 事物提交
conn.commit()
# 关闭数据库连接
cursor.close()
conn.close()

b'select * from student'


## 获取总条数

使用 cursor.rowcount 可以获得表中所有行总数；

In [13]:
# 获得连接
conn = psycopg2.connect(database="python", user="postgres", password="password", host="127.0.0.1", port="8032")
# 获得游标对象，一个游标对象可以对数据库进行执行操作
cursor = conn.cursor()
# sql语句 建表
sql ="""select * from student"""
# 执行语句
cursor.execute(sql)
count = cursor.rowcount
print(count)
# 事物提交
conn.commit()
# 关闭数据库连接
conn.close()

2


# 显示行号

使用 cursor.rownumber 可以显示当前查询sql获得数据的行号，每抓取一次光标的索引就会加1；

In [14]:
# 获得连接
conn = psycopg2.connect(database="python", user="postgres", password="password", host="127.0.0.1", port="8032")
# 获得游标对象，一个游标对象可以对数据库进行执行操作
cursor = conn.cursor()
# sql语句 建表
sql ="""select * from student """
# 执行语句
cursor.execute(sql)
row_1 = cursor.fetchone()
print(cursor.rownumber)
row_2 = cursor.fetchone()
print(cursor.rownumber)
# 事物提交
conn.commit()
# 关闭数据库连接
conn.close()

1
2


## 显示执行参数

使用 mogrify(operation[, parameters]) 能够显示执行语句的参数绑定结果，返回的是字符串形式；

In [15]:
# 获得连接
conn = psycopg2.connect(database="python", user="postgres", password="password", host="127.0.0.1", port="8032")
# 获得游标对象，一个游标对象可以对数据库进行执行操作
cursor = conn.cursor()
# sql语句 建表
sql ="""INSERT INTO student (num, name) VALUES (%s, %s)"""
params = (102, 'alphazz')
# 执行语句
result = cursor.mogrify(sql,params)
print(result.decode('UTF-8'))
cursor.execute(sql,params)
# 事物提交
conn.commit()
# 关闭数据库连接
conn.close()


INSERT INTO student (num, name) VALUES (102, 'alphazz')


## 附录
支持默认的类型转换如下,如果想要使用强制类型转换，详细的可以参照pgsql官网手册；

| Python | PostgreSQL |
| -- | -- |
| None | NULL |
| bool | bool |
| float | real，double |
| int，long | smallint，integer，bigint |
| Decimal | numeric |
| str，unicode | varchar，text |
| buffer，memoryview，bytearray，bytes，Buffer | protocol bytea |
| date | date |
| time | time,timetz |
| datetime | timestamp,timestamptz |
| timedelta | interval |
| list | ARRAY |
| tuple,namedtuple | Composite typesIN syntax |
| dict | hstore |
| Range | range |
| UUID | uuid |
| Anything | json |
| ipaddress | inet |